In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [8]:
transaction_data = pd.read_csv(r"C:\Users\mjklo\Downloads\archive\transactions_data.csv")

In [9]:
transaction_data["amount"] = (
    transaction_data["amount"]
    .replace(r"[\$,]", "", regex=True)  # removes $ and commas
    .astype(float)
)

In [11]:
transaction_data['transaction_month'] = pd.to_datetime(transaction_data['date']).dt.strftime('%Y-%m')
transaction_data['transaction_day'] = pd.to_datetime(transaction_data['date']).dt.strftime('%Y-%m-%d')

In [12]:
transaction_data['transaction_direction'] = 'Incoming'
transaction_data.loc[transaction_data["amount"] < 0,'transaction_direction'] = 'Outgoing'

In [13]:
amount_grouped = transaction_data.groupby(['client_id', 'transaction_month', 'transaction_direction']).amount.sum().reset_index()

In [14]:
# filtered = amount_grouped[amount_grouped['client_id'] == 300]
# # Pivot the data for plotting
# pivot_df = filtered.pivot(index="transaction_month", columns="transaction_direction", values="amount")

# # Plot as a line chart
# pivot_df.plot(kind="line", marker="o", figsize=(8, 5))
# plt.title("Incoming vs Outgoing Transactions per Month")
# plt.xlabel("Transaction Month")
# plt.ylabel("Amount ($)")
# plt.grid(True, linestyle="--", alpha=0.6)
# plt.tight_layout()
# plt.show()

In [15]:
# Make sure transaction_month is in datetime format
transaction_data["transaction_month"] = pd.to_datetime(transaction_data["transaction_month"])

# Find the most recent month in your dataset
last_month = transaction_data["transaction_month"].max()

# Compute the cutoff for the last x months
x = 6
cutoff = last_month - pd.DateOffset(months=x)

# Filter only last x months
last_x_months = transaction_data[transaction_data["transaction_month"] > cutoff]

# Compute monthly transaction volume (count of IDs)
transaction_volume = (
    last_x_months.groupby(["client_id", "transaction_month", "transaction_direction"])
    ["id"]
    .count()
)

# Compute average volume per client per direction across those x months
average_volume_lxm = (
    transaction_volume.groupby(["client_id", "transaction_direction"])
    .mean()
    .reset_index(name="average_volume_last_x_months")
)

# Pivot so Incoming/Outgoing become separate columns
volume_pivot = (
    average_volume_lxm
    .pivot(index="client_id", columns="transaction_direction", values="average_volume_last_x_months")
    .reset_index()
    .rename(columns={'Incoming': 'incoming_volume', 'Outgoing': 'outgoing_volume'})
)

# Fill missing values with 0 if a client has only one type of transaction
volume_pivot = volume_pivot.fillna(0)

In [16]:
amounts_grouped_lxm = last_x_months.groupby(['client_id', 'transaction_month', 'transaction_direction']).amount.sum().reset_index()

average_amounts_lxm = amounts_grouped_lxm.groupby(['client_id', 'transaction_direction']).amount.mean().reset_index(name='amount_mean_last_x_months')

# Pivot so Incoming/Outgoing become separate columns
amount_pivot = (
    average_amounts_lxm
    .pivot(index="client_id", columns="transaction_direction", values="amount_mean_last_x_months")
    .reset_index()
    .rename(columns={'Incoming': 'incoming_amount', 'Outgoing': 'outgoing_amount'})
)

# Fill missing values with 0 if a client has only one type of transaction
amount_pivot = amount_pivot.fillna(0)

In [17]:
# Compute monthly transaction volume (count of IDs)
client_affordability = (
    last_x_months.groupby(["client_id", "transaction_month"])
    ["amount"]
    .sum()
)

# Compute average volume per client per direction across those x months
average_affordability_lxm = (
    client_affordability.groupby(["client_id"])
    .mean()
    .reset_index(name="average_affordability_last_x_months")
)

# Compute average volume per client per direction across those x months
positive_balance_lxm = (
    client_affordability[client_affordability > 0].groupby(["client_id"])
    .count()
    .reset_index(name="positive_balance_lxm")
)

positive_balance_lxm['positive_balance_ratio'] = positive_balance_lxm['positive_balance_lxm'] / x

In [18]:
positive_balance_lxm

,client_id,positive_balance_lxm,positive_balance_ratio
0,0,6,1.0
1,1,6,1.0
2,2,6,1.0
3,3,6,1.0
4,4,6,1.0
...,...,...,...
1201,1994,6,1.0
1202,1995,6,1.0
1203,1996,6,1.0
1204,1997,6,1.0


In [11]:
# Sort by client and date
df = transaction_data.sort_values(by=["client_id", "date"])

# Ensure 'date' is datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Compute days since last transaction for each client + direction
df["days_since_last"] = (
    df.groupby(["client_id", "transaction_direction"])["date"]
      .diff()  # timedelta between current and previous transaction
      .dt.days  # convert to number of days
)

# Compute average transaction frequency (mean days between transactions)
transaction_frequency = (
    df.groupby(["client_id", "transaction_direction"])["days_since_last"]
      .mean()
      .reset_index()
)

# Pivot so Incoming/Outgoing become separate columns
freq_pivot = (
    transaction_frequency
    .pivot(index="client_id", columns="transaction_direction", values="days_since_last")
    .reset_index()
    .rename(columns={'Incoming': 'incoming_frequency', 'Outgoing': 'outgoing_frequency'})
)

# Fill missing values with 0 if a client has only one type of transaction
freq_pivot = freq_pivot.fillna(0)


In [12]:
# Ensure 'transaction_day' is datetime type
transaction_data["transaction_day"] = pd.to_datetime(transaction_data["transaction_day"], errors="coerce")

# Get the first (earliest) transaction date per client
time_on_book = transaction_data.groupby("client_id").agg({'transaction_day': 'min'}).reset_index()

# Calculate months on book (from first transaction to now)
time_on_book["months_on_book"] = (datetime.now() - time_on_book["transaction_day"]).dt.days / 30.44

In [13]:
# Compute variance per client and direction
amount_variance = (
    amounts_grouped_lxm.groupby(["client_id", "transaction_direction"])
    .amount.var()
)

# Pivot the Series so Incoming/Outgoing become separate columns
var_pivot = (
    amount_variance
    .unstack(level="transaction_direction")
    .reset_index()
    .rename(columns={'Incoming': 'incoming_variance', 'Outgoing': 'outgoing_variance'})
)

# Fill missing values with 0 if a client has only one type of transaction
var_pivot = var_pivot.fillna(0)

In [14]:
cumulative_balance = amount_grouped.groupby("client_id").amount.sum().rename("cumulative_balance").reset_index()

In [15]:
feature_table = pd.merge(time_on_book[["client_id", "months_on_book"]], average_affordability_lxm, on="client_id")\
  .merge(freq_pivot, on="client_id")\
  .merge(var_pivot, on="client_id")\
  .merge(cumulative_balance, on="client_id")\
  .merge(volume_pivot, on="client_id")\
  .merge(amount_pivot, on="client_id")\
  .merge(positive_balance_lxm[['client_id', 'positive_balance_ratio']], on='client_id', how='left')

In [16]:
feature_table['affordability_buffer'] = feature_table['cumulative_balance']/feature_table['average_affordability_last_x_months']
feature_table['direction_ratio'] = -feature_table['outgoing_amount']/feature_table['incoming_amount']

In [17]:
income_last_x_months = (
    last_x_months[last_x_months['transaction_direction'] == 'Incoming'].groupby(["client_id", "transaction_month"])
    ["amount"]
    .sum()
)
income_affordability_comparison = pd.merge(income_last_x_months, feature_table[['client_id', 'average_affordability_last_x_months']], on='client_id', how='left')
income_affordability_comparison['income_greater_than_boundary'] = income_affordability_comparison['amount'] > income_affordability_comparison['average_affordability_last_x_months']

boundary_grouped = income_affordability_comparison.groupby('client_id')['income_greater_than_boundary'].mean().reset_index(name='boundary_exceedance_ratio')
boundary_grouped['target'] = (boundary_grouped['boundary_exceedance_ratio'] < 0.5).astype(int)

In [18]:
boundary_grouped.target.sum()/len(boundary_grouped)

np.float64(0.05223880597014925)

In [19]:
feature_table = pd.merge(feature_table, boundary_grouped[['client_id', 'target']], on='client_id', how='left')

In [20]:
feature_table.to_csv(r"C:\Users\mjklo\Downloads\feature_table.csv", index=False)

In [21]:
feature_table

,client_id,months_on_book,average_affordability_last_x_months,incoming_frequency,outgoing_frequency,incoming_variance,outgoing_variance,cumulative_balance,incoming_volume,outgoing_volume,incoming_amount,outgoing_amount,positive_balance_ratio,affordability_buffer,direction_ratio,target
0,0,189.553219,4982.388333,0.029525,5.156151,6.730129e+05,63290.400000,625799.67,101.666667,5.500000,5602.388333,-620.000000,1.0,125.602347,0.110667,0
1,1,189.553219,2948.665000,0.043706,24.617021,1.639791e+06,173804.916667,336187.37,82.500000,2.250000,3265.498333,-475.250000,1.0,114.013416,0.145537,1
2,2,189.553219,2396.293333,0.046422,6.585799,2.996908e+05,18802.966667,291534.27,87.833333,4.833333,2783.460000,-387.166667,1.0,121.660510,0.139095,0
3,3,189.520368,2496.531667,0.197948,66.415094,2.710008e+05,97571.700000,280685.46,51.333333,1.600000,2757.198333,-312.800000,1.0,112.430162,0.113448,0
4,4,189.553219,5169.085000,0.012374,4.995427,4.076205e+05,140812.266667,595722.36,124.166667,6.000000,5735.751667,-566.666667,1.0,115.247159,0.098796,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201,1994,178.580815,4506.855000,0.069759,5.473492,1.286872e+06,123374.666667,430112.77,81.000000,5.000000,5142.188333,-635.333333,1.0,95.435236,0.123553,0
1202,1995,189.553219,3261.378333,0.031264,6.088561,7.551924e+05,79785.066667,416625.38,93.166667,4.666667,3684.711667,-423.333333,1.0,127.745185,0.114889,0
1203,1996,189.553219,2705.293333,0.160760,5.824253,3.307485e+05,28398.400000,343940.81,55.000000,4.333333,3131.293333,-426.000000,1.0,127.136235,0.136046,0
1204,1997,189.520368,5550.248333,0.057542,5.332795,2.886756e+06,130475.200000,592162.39,88.333333,5.000000,6075.248333,-525.000000,1.0,106.691152,0.086416,0
